# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [9]:
# imports
from IPython.display import Markdown, display, update_display
import openai
from openai import OpenAI


In [10]:
# constants
models = {
    'MODEL_GPT': 'gpt-4o-mini',
    'MODEL_LLAMA': 'llama3.2'
}

# To use ollama using openai API (ensure that ollama is running on localhost)
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

def model_choices(model):
    if model in models:
        return models[model]
    else:
        raise ValueError(f"Model {model} not found in models dictionary")

def get_model_api(model='MODEL_GPT'):
    if model == 'MODEL_GPT':
        return openai, model_choices(model)
    elif model == 'MODEL_LLAMA':
        return ollama_via_openai, model_choices(model)
    else:
        raise ValueError(f"Model {model} not found in models dictionary")


In [12]:
# set up environment

system_prompt = """ You are an AI assistant helping a user find information about a product. 
The user asks you a technical question about code, and you provide a response with code snippets and explanations."""

def stream_brochure(question, model):
    api, model_name = get_model_api(model)
    stream = api.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": question}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)



In [13]:
# Here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [ ]:
# Get the model of your choice (choices appeared below) to answer, with streaming 

"""models = {
    'MODEL_GPT': 'gpt-4o-mini',
    'MODEL_LLAMA': 'llama3.2'
}"""

stream_brochure(question,'MODEL_LLAMA')

**Understanding the Code Snippet**

This Python code snippet uses a combination of built-in functions, dictionary iteration, and generator expressions to extract and yield author names from a list of `Book` objects.

Here's a breakdown:

1. **Dictionary Iteration**: The expression `for book in books if book.get("author")`
   - Iterates over each element (`book`) in the container `books`.
   - Filters out elements whose `'author'` key does not have a value (i.e., `None`, `False`, or an empty string). This leaves only dictionaries with author information.

2. **Dictionary Access**: The expression `{book.get("author") for book in books if book.get("author")}`
   - Uses dictionary membership testing to access only the values associated with the `'author'` key.
   - If the value is not found or is considered false, it's skipped in this particular case.

3. **Generator Expression**: This generates an iterator that iterates over the filtered author names.
   - Yields each author name (i.e., a single `'name'` from the book dictionary) on demand.
   - Since these are generator expressions, they use memory less than equivalent Python lists and also create results on-demand.

4. **`yield from`**: This statement takes the generator expression as an argument and uses it to generate a nested iterator structure.
   - It essentially "decompresses" the single level of nested iterator created by `list(iter(x))`, allowing for simpler use cases and potentially significant efficiency improvements for more complex structures where every value must be iterated, while in the latter case just the first item per iterable in the outer expression's sequence needs to actually be yielded into result stream.
   - By "yielding" a nested iterator (the generator expression), we can simplify code by avoiding repetitive structure like `for book, book_author in zip(iterating over), ...` or list creation.

**Example Use Case**

In this hypothetical example:

# Example Book objects
class Book:
    def __init__(self, author, title):
        self.author = author  # str
        self.title = title

books = [
    {"author": "John Doe", "title": f"Book 1 by John Doe"},
    {"author": None, "title": f"Book 2 without Author"},
    {"author": "Jane Smith", "title": f"Book 3 by Jane Smith"}
]

# The given expression to extract and yield author names
for author in yield from {book.get("author") for book in books if book.get("author")}:

    print(author) 

In this code snippet, printing the extracted authors would output `John Doe`, `Jane Smith` (since only dictionaries with author information pass the filtering test).

Please modify it like as you wish and use `yield from` along with dictionary iteration, list comprehension or generator expression if needed, and explain what purpose your version has.